In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import pickle

In [ ]:
#Dataframe
df = pd.read_csv("../../data/raw/milanuncios.com.csv")
df.drop(columns=["Unnamed: 0"], inplace=True)

#Sentence Transformer
model_sentence = SentenceTransformer("all-MiniLM-L6-v2")

#Classifier model for text
with open('../modeling/trained/modelo_texto.pkl', 'rb') as f:
    modelo_texto = pickle.load(f)


In [19]:
top_30_coches = df['car_type'].value_counts().nlargest(30).index
top_30_coches

Index(['TESLA - MODEL 3', 'CUPRA - FORMENTOR', 'CUPRA - LEON',
       'PORSCHE - CAYENNE', 'AUDI - A3', 'AUDI - A4', 'BMW - X2',
       'BMW - SERIE', 'TESLA - MODEL Y', 'PORSCHE - PANAMERA',
       'TESLA - MODEL S', 'PORSCHE - TAYCAN', 'AUDI - A6', 'PORSCHE - 911',
       'BMW - X1', 'PORSCHE - MACAN', 'TESLA - MODEL X', 'CUPRA - TERRAMAR',
       'CUPRA - ATECA', 'AUDI - A5', 'BMW - X3', 'BMW - X5', 'BMW - COMPACT',
       'PORSCHE - BOXSTER', 'CUPRA - BORN', 'BMW - X6', 'AUDI - TT',
       'AUDI - A1', 'AUDI - Q3', 'BMW - Z4'],
      dtype='object', name='car_type')

In [18]:
df_top30 = df[df['car_type'].isin(top_30_coches)].copy()
df_top30

,car_type,year,price,km,description,photo_url
0,TESLA - MODEL 3,2022,32.990 €,67.405 kms,Precio al contado: 32.990€ precio alternativo ...,https://images.milanuncios.com/api/v1/ma-ad-me...
1,TESLA - MODEL S,2019,46.500 €,37.500 kms,Ocasionplus - 12000 coches - 100 centros ocasi...,https://images.milanuncios.com/api/v1/ma-ad-me...
2,TESLA - MODEL X,2018,37.490 €,140.261 kms,Precio al contado: 37.490€ precio alternativo ...,https://images.milanuncios.com/api/v1/ma-ad-me...
3,TESLA - MODEL Y,2023,36.900 €,33.000 kms,"AÑO 2023 GARANTÍA TESLA Motor dual, tracción 4...",https://images.milanuncios.com/api/v1/ma-ad-me...
4,TESLA - MODEL Y,2023,44.990 €,29.000 kms,Interesados llamar al 680684213 -Único propiet...,https://images.milanuncios.com/api/v1/ma-ad-me...
...,...,...,...,...,...,...
2020,CUPRA - LEON,2025,34.305 €,0 kms,"Precio pago al contado: 34.305 €, consultar ca...",https://images.milanuncios.com/api/v1/ma-ad-me...
2021,CUPRA - LEON,2025,28.885 €,0 kms,"Precio pago al contado: 28.885 €, consultar ca...",https://images.milanuncios.com/api/v1/ma-ad-me...
2022,CUPRA - FORMENTOR,2025,49.400 €,3.500 kms,Cupra formentor vz atendemos solo con cita pre...,https://images.milanuncios.com/api/v1/ma-ad-me...
2024,CUPRA - LEON,2025,44.600 €,0 kms,"Exclusiva oferta online en talleres rubens, co...",https://images.milanuncios.com/api/v1/ma-ad-me...


In [25]:
df_top30.isna().sum()

car_type       0
year           0
price          0
km             0
description    0
photo_url      0
dtype: int64

In [ ]:
conteo_cv = df_top30['description'].str.contains(r'\bcv\b', case=False, na=False).sum()
print(f"Número de descripciones que contienen 'cv': {conteo_cv}")

Número de descripciones que contienen 'cv': 212


In [ ]:
descripciones = df_top30["description"].fillna("")

In [ ]:
desc_encoded = model_sentence.encode(descripciones)

predicciones = modelo_texto.predict(desc_encoded)
df_top30["prediccion"] = predicciones

In [ ]:
df.to_csv('../transformed/milanuncios_predi.csv', index=False)